### README

* Timestamp
  * Convert and merge timestamps.
* Account No & Balance & Amount
  * Eliminate errors in transaction information of a single user (based on the amount of each transaction and the latest balance).
  * For a single user, find the vacancies outside the error range of the transaction information (balance, amount), and fill them with\
  unowned (Account No is invalid) information within the corresponding time range.
  * If there are still gaps after transaction information has been filled, consider adding new rows.
  * If there are still gaps after user information has been filled, consider changing it to 0 directly.
* Third-party Account Classification
  * Third-party Accounts with valid 'Names' will be classified according to their business content based on 'class2.xlsx', as follows:
    * 1:  Technology & Cultural Development
    * 2:  Fashion Trend
    * 3:  Lifestyle & Entertainment Crafts
    * 4:  Health & Living Services
    * 5:  Dining & Leisure
    * 6:  Comprehensive Retail Market
    * 7:  Active Lifestyle & Fitness
    * 8:  Financial Services & Accommodation
  * Third-party Accounts with only valid 'Account No' will be classified as 'Personal'.
  * Third-party Accounts with no valid information will be classified as 'Unknown'.
  * Missing 'Accound No' information will be filled with 0, and missing 'Name' information will be filled according to its classification.
* Save as CSV
  * Export the DataFrame locally.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('simulated_transaction_2024.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230596 entries, 0 to 230595
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Date                    230367 non-null  object 
 1   Timestamp               230345 non-null  object 
 2   Account No              230372 non-null  float64
 3   Balance                 230350 non-null  float64
 4   Amount                  230387 non-null  float64
 5   Third Party Account No  6832 non-null    float64
 6   Third Party Name        223517 non-null  object 
dtypes: float64(4), object(3)
memory usage: 12.3+ MB
None


### Timestamp

In [3]:
df['Timestamp']=pd.to_datetime(df['Date']+' '+df['Timestamp'],format='%d/%m/%Y %H:%M')
df=df.drop('Date',axis=1)

In [4]:
while len(df[df['Timestamp'].isnull()==False])!=230596:
    temp_df=pd.DataFrame()
    temp_df['before']=df['Timestamp'].shift(1)
    temp_df['after']=df['Timestamp'].shift(-1)
    temp_df['mean']=temp_df.mean(axis=1,skipna=True,numeric_only=False)
    df['Timestamp']=df['Timestamp'].fillna(temp_df.mean)

### Account No & Balance & Amount

In [5]:
print(df[df.Amount>.001].Amount.abs().min())

10.0


In [6]:
total_T=0
total_F=0

In [7]:
users=list(set(df['Account No'].dropna()))
nan_df=df[np.isnan(df['Account No'])==True]
nan_users=np.array([nan_df.index.values,nan_df.Balance.values,nan_df.Amount.values])
min_vacancy=1

for user in users:
    temp_df=df[df['Account No']==user].iloc[::-1]
    temp_df=temp_df.round(2)
    loop=True
    
    # fill nan
    for t in range(2):
        while len(temp_df[np.isnan(temp_df.iloc[:,2])==True])+len(temp_df[np.isnan(temp_df.iloc[:,3])==True])>0:
            for index in range(len(temp_df)-1):
                if pd.isna(temp_df.iloc[index,2])==True:
                    temp_df.iloc[index,2]=temp_df.iloc[index-1,2]-temp_df.iloc[index-1,3]
                if pd.isna(temp_df.iloc[index,3])==True:
                    temp_df.iloc[index,3]=temp_df.iloc[index,2]-temp_df.iloc[index+1,2]
        df[df['Account No']==user]=temp_df.iloc[::-1]
        
        # find and fit
        
        while loop==True:
            loop=False
            balance=[temp_df.iloc[0,2]]
            amounts=temp_df.iloc[:,3].tolist()
            for index in range(len(temp_df)-1):
                balance.append(balance[-1]-amounts[index])
                diff=balance[-1]-temp_df.iloc[index+1,2]
                if abs(diff)>min_vacancy:
                    after=temp_df.index[index]
                    before=temp_df.index[index+1]
                    
                    a=np.argmin(nan_users[0]<after)
                    b=np.argmax(nan_users[0]>before)
                    
                    if b<a:
                        try:
                            c1=np.nanargmin(np.abs(nan_users[1,b:a]-balance[-1]))
                            c2=np.nanargmin(np.abs(nan_users[2,b:a]-diff))
                        except:
                            c1=np.argmin(np.abs(nan_users[1,b:a]-balance[-1]))
                            c2=np.argmin(np.abs(nan_users[2,b:a]-diff))
                        finally:
                            c=np.intersect1d(c1,c2)+b
                            if c.size==0:
                                if np.where(nan_users[1,b:a]==nan_users[1,c1+b])[0].size>1 and \
                                np.where(nan_users[2,b:a]==nan_users[2,c2+b])[0].size>1:
                                    print('Problem')
                                elif np.where(nan_users[1,b:a]==nan_users[1,c1+b])[0].size>1:
                                    c=c2+b
                                else:
                                    c=c1+b
                    else:
                        c=b
                    
                    if c.size>0 and abs(nan_users[1,c]-balance[-1])<1 and abs(nan_users[2,c]-diff)<1:
                        total_T+=1
                        df.iloc[int(nan_users[0,c]),1]=user
                        temp_df.loc[int(nan_users[0,c])]=df.iloc[int(nan_users[0,c])]
                        temp_df.sort_index(ascending=False,inplace=True)
                        loop=True
                    elif np.isnan(nan_users[1:2,c]).sum()>0:
                        #nan=7796.92 -19.19
                        total_T+=1
                        df.iloc[int(nan_users[0,c]),1]=user
                        temp_df.loc[int(nan_users[0,c])]=df.iloc[int(nan_users[0,c])]
                        temp_df.sort_index(ascending=False,inplace=True)
                    else:
                        total_F+=1
                        print('User '+str(int(user))+"'s transaction record cannot be completed!!!")
                        
                        #       If there are still failed attempts in the output,
                        #  the following code will be improved and used to add new rows
                        # to resolve transactions that cannot be filled with existing rows.
                        
                        #add_df=pd.DataFrame({'Timestamp':[temp_df.iloc[index+1,0]+(temp_df.iloc[index,0]-temp_df.iloc[index+1,0])/2],
                                             #'Account No':[user],
                                             #'Balance':[balance[-1]],
                                             #'Amount':[diff],
                                             #'Third Party Account No':[np.nan],
                                             #'Third Party Name':[np.nan]
                                            #})
                        #cat_list=[temp_df.iloc[:index+1],add_df,temp_df.iloc[index+1:]]
                        #pd.set_option('display.max_rows',len(temp_df)+1)
                        #display(pd.concat(cat_list))
                        #pd.reset_option('display.max_rows')
                    
                    np.delete(nan_users,c,1)
                    break
                    
                temp_df.iloc[:index+2,2]=balance
                
    loop=False
    
print('========== The process is over ==========')
print('Successful: '+str(total_T))
print('Failed: '+str(total_F))

========== The process is over ==========
Successful: 222
Failed: 0


### Third-party Account Classification

In [8]:
EN_dict=pd.read_excel('class2.xlsx').iloc[1:9,6].str.replace(' ','').to_dict()

class_df=pd.read_excel('class2.xlsx').iloc[:,[0,2]]
class_df.columns=['Name','Number']
class_df['Name']=class_df['Name'].str.replace("' ",'')
class_df=class_df.set_index('Name')
class_df['Class']=class_df.replace({'Number':EN_dict})
class_dict=class_df.to_dict()

df['Classification Number']=df.replace({'Third Party Name':class_dict['Number']}).iloc[:,5]
df['Classification Title']=df.replace({'Third Party Name':class_dict['Class']}).iloc[:,5]

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230596 entries, 0 to 230595
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Timestamp               230596 non-null  object 
 1   Account No              230594 non-null  float64
 2   Balance                 230596 non-null  float64
 3   Amount                  230596 non-null  float64
 4   Third Party Account No  6832 non-null    float64
 5   Third Party Name        223517 non-null  object 
 6   Classification Number   223517 non-null  object 
 7   Classification Title    223517 non-null  object 
dtypes: float64(4), object(4)
memory usage: 14.1+ MB
None


In [10]:
null_list=[]
for col in df.columns:
    null_list.append(list(df[df[col].isna()==True].index.values))
unknown_TP=list(set(null_list[4]).intersection(set(set(null_list[5]))))
df.iloc[unknown_TP,[5,7]]=df.iloc[unknown_TP,[5,7]].fillna('Unknown')

df.iloc[:,[1,4,6]]=df.iloc[:,[1,4,6]].fillna(0)
df.iloc[:,[5,7]]=df.iloc[:,[5,7]].fillna('Personal')

### Save as CSV

In [11]:
df.to_csv('fixed_simulated_transaction_2024.csv')